# Water budget

- This notebook is intended for constructing water balance (monthly and daily) and cumulative water deficit (monthly) datasets.

# Initial setup

In [1]:
# Load packages.
import datetime
import glob
import os

import numpy as np
import xarray as xr

%matplotlib inline

# Load monthly data

## Precipitation and evaporation

In [2]:
FILES1 = sorted(
    glob.glob("/media/alex/ALEXDATA/data_sets/ERA_INTERIM/total_precipitation_mmeans/*grb")) 

FILES2 = sorted(
    glob.glob("/media/alex/ALEXDATA/data_sets/ERA_INTERIM/evaporation_mmeans/*grb")) 

TP = xr.open_mfdataset(FILES1, engine="cfgrib")
E = xr.open_mfdataset(FILES2, engine="cfgrib")

# Guarantee time order.
TP = TP.sortby(TP.time)
E = E.sortby(E.time)

# Monthly water balance

First we create a variable called water balance ($WB$) which is based on the difference between total precipitation ($TP$) and evaporation ($E$):

$$
WB = TP - E
$$

Please note the fliped sign in evaporation on the above cell.This is so because of ERA interim definition of evaporation, as you can see in [evaporation parameter details](https://apps.ecmwf.int/codes/grib/param-db?id=182). Also, remember that evaporation is the accumulated amount of water that has evaporated from the Earth's surface, **including a simplified representation of transpiration (from vegetation)**, into vapour in the air above.

In [3]:
# Create water balance variable as a xarray DataSet object. 
WB = TP + E

# Create the corresponding variable with some attributes.
WB = WB.assign(variables={"wb": TP.tp + E.e})
WB.wb.attrs["calculation"] = "Made by Alex Araujo at " + \
                             datetime.datetime.now().strftime("%Y-%m-%d")
WB.wb.attrs["long_name"] = "Water balance"
WB.wb.attrs["units"] = "m"
WB.wb

<xarray.DataArray 'wb' (time: 480, latitude: 241, longitude: 480)>
dask.array<shape=(480, 241, 480), dtype=float32, chunksize=(12, 241, 480)>
Coordinates:
    number      int64 0
    step        timedelta64[ns] 12:00:00
    surface     int64 0
  * latitude    (latitude) float64 90.0 89.25 88.5 87.75 ... -88.5 -89.25 -90.0
  * longitude   (longitude) float64 0.0 0.75 1.5 2.25 ... 357.8 358.5 359.2
  * time        (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2018-12-01
    valid_time  (time) datetime64[ns] dask.array<shape=(480,), chunksize=(12,)>
Attributes:
    calculation:  Made by Alex Araujo at 2019-06-05
    long_name:    Water balance
    units:        m

In [4]:
# Export data. Each file is one year data.
for year in np.unique(WB.time.dt.year.values):
    
    # Data for that year.
    DS = WB.sel(time=str(year))
    
    # Name for this file
    name = "wb_mmeans_" + str(year) + ".nc"
    
    # File name path.
    folder = "/media/alex/ALEXDATA/data_sets/ERA_INTERIM/water_balance_mmeans/"
    outpath = folder + name
    
    # Continue only if data file absdoes not exist yet.
    if not os.path.isfile(outpath):  
        
        # Message.
        print("Creating", name, "...")
    
        # Export data as netcdf file.
        DS.to_netcdf(outpath)
        
    else:
        
        # Message.
        print("Warning:", name, "has already been created!")

# Monthly water deficit

In [5]:
# Calculate water deficit. First, extract water balance data as numpy array.
WB_data = WB.wb.values

# Initialize numpy array for water deficit data as zeros.
WD_data = np.zeros_like(WB_data)

# First time step.
WD_data[0] = np.minimum(WB_data[0], 0)

# Loop over the remaining time steps:
for n in range(1, WB_data.shape[0]):
    
    # Deficit occurs in two consecutive days.
    deficit = WD_data[n - 1] + WB_data[n] < 0
      
    # Accumulate negative number for the current deficit where deficit==True.
    # Otherwise, the numpy array gets zero at the location
    WD_data[n] = np.where(deficit, WD_data[n - 1] + WB_data[n], 0)

# First we create a xarray DataArray object that will contain data for the subsequent DataSet
# object from xarray.
wd = xr.DataArray(data=WD_data, 
                  coords={"time": WB.time, 
                          "longitude": WB.longitude,
                          "latitude": WB.latitude},
                  dims=("time", "latitude", "longitude")
                 )

    # Now we can create DataSet object for water deficit variable.
    WD = xr.Dataset(data_vars={"wd": wd})
    WD.wd.attrs["calculation"] = "Made by Alex Araujo at " + \
                                 datetime.datetime.now().strftime("%Y-%m-%d")
    WD.wd.attrs["long_name"] = "Accumulative water deficit"
    WD.wd.attrs["units"] = "m"

WD

<xarray.Dataset>
Dimensions:    (latitude: 241, longitude: 480, time: 480)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2018-12-01
  * longitude  (longitude) float64 0.0 0.75 1.5 2.25 ... 357.0 357.8 358.5 359.2
  * latitude   (latitude) float64 90.0 89.25 88.5 87.75 ... -88.5 -89.25 -90.0
Data variables:
    wd         (time, latitude, longitude) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [6]:
# Export data. Each file is one year data.
for year in np.unique(WD.time.dt.year.values):
    
    # Data for that year.
    DS = WD.sel(time=str(year))
    
    # Name for this file
    name = "wd_mmeans_" + str(year) + ".nc"
    
    # File name path.
    folder = "/media/alex/ALEXDATA/data_sets/ERA_INTERIM/water_deficit_mmeans/"
    outpath = folder + name
    
    # Continue only if data file absdoes not exist yet.
    if not os.path.isfile(outpath):  
        
        # Message.
        print("Creating", name, "...")
    
        # Export data as netcdf file.
        DS.to_netcdf(outpath)
        
    else:
        
        # Message.
        print("Warning:", name, "has already been created!")

# Load daily data

## Precipitation and evaporation

In [7]:
FILES1 = sorted(
    glob.glob("/media/alex/ALEXDATA/data_sets/ERA_INTERIM/total_precipitation_daily/*nc")) 

FILES2 = sorted(
    glob.glob("/media/alex/ALEXDATA/data_sets/ERA_INTERIM/evaporation_daily/*grb")) 

TP = xr.open_mfdataset(FILES1)
E = xr.open_mfdataset(FILES2, engine="cfgrib")

# Guarantee time order.
TP = TP.sortby(TP.time)
E = E.sortby(E.time)

# Put data in the same grid (This is not necessary. Xarray does it automatically!).
TP, E = xr.align(TP, E)

# Daily water balance

In [8]:
# Create water balance variable as a xarray DataSet object. 
WB = TP + E

# Create the corresponding variable with some attributes.
WB = WB.assign(variables={"wb": TP.tp + E.e})
WB.wb.attrs["calculation"] = "Made by Alex Araujo at " + \
                             datetime.datetime.now().strftime("%Y-%m-%d")
WB.wb.attrs["long_name"] = "Water balance"
WB.wb.attrs["units"] = "m"
WB.wb

<xarray.DataArray 'wb' (time: 29219, latitude: 241, longitude: 480)>
dask.array<shape=(29219, 241, 480), dtype=float32, chunksize=(61, 241, 480)>
Coordinates:
  * time        (time) datetime64[ns] 1979-01-01T12:00:00 ... 2018-12-31T12:00:00
  * longitude   (longitude) float32 0.0 0.75 1.5 2.25 ... 357.75 358.5 359.25
  * latitude    (latitude) float32 90.0 89.25 88.5 87.75 ... -88.5 -89.25 -90.0
    number      int64 0
    step        timedelta64[ns] 12:00:00
    surface     int64 0
    valid_time  (time) datetime64[ns] dask.array<shape=(29219,), chunksize=(61,)>
Attributes:
    calculation:  Made by Alex Araujo at 2019-06-05
    long_name:    Water balance
    units:        m

In [9]:
# Export data. Each file is one month data.
for year in np.unique(WB.time.dt.year.values):
    for month in np.unique(WB.time.dt.month.values):

        # Data for that month.
        DS = WB.sel(time=str(year) + "-" + str(month))
    
        # Name for this file
        name = "wb_daily_" + str(year) + "_" + str("{:02d}".format(month)) + ".nc"
    
        # File name path.
        folder = "/media/alex/ALEXDATA/data_sets/ERA_INTERIM/water_balance_daily/"
        outpath = folder + name
    
        # Continue only if data file absdoes not exist yet.
        if not os.path.isfile(outpath):  
        
            # Message.
            print("Creating", name, "...")
    
            # Export data as netcdf file.
            DS.to_netcdf(outpath)
        
        else:
        
            # Message.
            print("Warning:", name, "has already been created!")